Flask Application

In [23]:
%pip install flask opencv-python-headless supervision roboflow numpy pillow 

In [1]:
%pip install roboflow supervision flask opencv-python-headless numpy Pillow


In [2]:
from roboflow import Roboflow
import supervision as sv
import cv2
import numpy as np
from flask import Flask, request, jsonify
import base64
import io
from PIL import Image
from werkzeug.serving import run_simple

# Initialize Roboflow with your API key
rf = Roboflow(api_key="8Qa7KjefjGCaCBRyRn7D")
project = rf.workspace().project("tissue-v2")
model = project.version(4).model  # Load model version 4

app = Flask(__name__)

def resize_image(image, max_size=800):
    """Resize the image to a maximum size while maintaining aspect ratio."""
    height, width = image.shape[:2]
    if max(height, width) > max_size:
        scale = max_size / max(height, width)
        new_width = int(width * scale)
        new_height = int(height * scale)
        image = cv2.resize(image, (new_width, new_height))
    return image

@app.route('/infer', methods=['POST'])
def infer():
    try:
        # Load the image from the request
        file = request.files['file']
        image = cv2.imdecode(np.frombuffer(file.read(), np.uint8), cv2.IMREAD_COLOR)
        
        # Resize the image to avoid size issues
        image = resize_image(image)

        # Save the resized image to a temporary file
        temp_image_path = "temp_image.jpg"
        cv2.imwrite(temp_image_path, image)

        # Perform inference with Roboflow using the image file
        response = model.predict(temp_image_path, hosted=True).json()

        # Parse the response into Supervision Detections format
        detections = sv.Detections.from_roboflow(response)

        # Annotate the image with bounding boxes
        box_annotator = sv.BoxAnnotator()
        annotated_image = box_annotator.annotate(scene=image, detections=detections)

        # Convert the annotated image to base64
        _, annotated_buffer = cv2.imencode('.jpg', annotated_image)
        annotated_base64 = base64.b64encode(annotated_buffer).decode('utf-8')

        # Return the base64-encoded annotated image as JSON
        return jsonify({"annotated_image": annotated_base64})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Run the server with Werkzeug for Jupyter compatibility
if __name__ == '__main__':
    run_simple('0.0.0.0', 5000, app)


loading Roboflow workspace...
loading Roboflow project...


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.201.73:5000
Press CTRL+C to quit
127.0.0.1 - - [14/Feb/2025 22:38:01] "POST /process HTTP/1.1" 404 -
